In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-06 13:52:53,656 INFO: Initializing external client
2025-03-06 13:52:53,658 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 13:52:54,769 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681
Fetching data from 2025-02-05 18:52:53.654361+00:00 to 2025-03-06 17:52:53.654361+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (4.71s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-05 19:00:00+00:00,2,0
1,2025-02-05 20:00:00+00:00,2,0
2,2025-02-05 21:00:00+00:00,2,0
3,2025-02-05 22:00:00+00:00,2,0
4,2025-02-05 23:00:00+00:00,2,0
...,...,...,...
175135,2025-03-06 13:00:00+00:00,263,125
175136,2025-03-06 14:00:00+00:00,263,112
175137,2025-03-06 15:00:00+00:00,263,121
175138,2025-03-06 16:00:00+00:00,263,108


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 181175
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         175140 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  175140 non-null  int32                  
 2   rides               175140 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 175140 entries, 0 to 181175
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         175140 non-null  datetime64[us]
 1   pickup_location_id  175140 non-null  int32         
 2   rides               175140 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,99,65,6,168,44,30,30,24,15,71,...,26,39,42,46,52,58,81,140,114,2025-03-05 19:00:00
1,0,2,0,0,0,0,0,0,2,0,...,2,0,3,1,1,1,2,1,165,2025-03-05 19:00:00
2,1,0,3,1,4,0,1,2,0,1,...,0,0,0,1,0,4,1,0,93,2025-03-05 19:00:00
3,112,11,127,41,81,35,48,104,95,88,...,88,100,108,136,153,181,215,212,263,2025-03-05 19:00:00
4,0,0,0,0,0,0,2,0,0,0,...,0,0,0,1,0,0,0,0,241,2025-03-05 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,1,0,0,0,0,0,0,0,0,1,...,0,0,0,0,0,0,0,0,94,2025-03-05 19:00:00
248,1,1,0,0,0,1,2,1,0,0,...,1,0,0,2,0,0,0,0,174,2025-03-05 19:00:00
249,0,0,0,0,0,1,0,0,0,2,...,0,1,1,0,0,0,0,0,185,2025-03-05 19:00:00
250,2,0,2,1,3,2,0,1,1,0,...,2,1,0,0,0,2,3,4,37,2025-03-05 19:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-06 13:53:05,836 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 13:53:05,845 INFO: Initializing external client
2025-03-06 13:53:05,846 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 13:53:06,933 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681
Fetching data from 2025-02-05 18:53:05.836470+00:00 to 2025-03-06 17:53:05.836470+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.08s) 


In [12]:
current_date

Timestamp('2025-03-06 18:53:05.836470+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-05 19:00:00
1,0,0,0,0,0,0,0,0,0,1,...,1,0,0,0,0,0,0,0,3,2025-03-05 19:00:00
2,6,5,4,4,4,0,3,1,0,1,...,3,5,4,3,5,7,18,20,4,2025-03-05 19:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,6,2025-03-05 19:00:00
4,2,1,1,0,1,2,1,1,0,0,...,2,1,2,2,8,4,1,8,7,2025-03-05 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
247,0,0,0,0,0,0,0,0,0,1,...,1,0,0,1,0,0,0,0,259,2025-03-05 19:00:00
248,1,0,1,2,0,2,1,0,2,0,...,1,2,0,4,2,0,0,2,260,2025-03-05 19:00:00
249,43,37,20,31,10,4,1,0,0,2,...,28,35,43,42,43,59,40,50,261,2025-03-05 19:00:00
250,78,35,38,13,8,2,2,1,0,6,...,109,105,97,100,134,169,178,163,262,2025-03-05 19:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-06 13:53:17,550 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-06 13:53:20,564 INFO: Initializing external client
2025-03-06 13:53:20,565 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-06 13:53:21,633 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215681


In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,-0.0
1,3,0.0
2,4,3.0
3,6,-0.0
4,7,1.0
...,...,...
247,259,0.0
248,260,1.0
249,261,23.0
250,262,27.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([236, 237, 138, 132, 107, 249, 234,  48, 230,  68])